In [1]:
from dotenv import load_dotenv
import os 
from langchain_openai import AzureChatOpenAI
from langchain_core.output_parsers import StrOutputParser

load_dotenv('env', override=True)
AZURE_OPENAI_API_KEY = os.getenv('AZURE_OPENAI_API_KEY')
END_POINT=os.getenv('END_POINT')
MODEL_NAME=os.getenv('MODEL_NAME')
print(AZURE_OPENAI_API_KEY[:10])
print(MODEL_NAME)

AZURE_OPENAI_EMB_API_KEY = os.getenv('AZURE_OPENAI_EMB_API_KEY')
EMB_END_POINT=os.getenv('EMB_END_POINT')
EMB_MODEL_NAME=os.getenv('EMB_MODEL_NAME')

os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGSMITH_API_KEY')
os.environ['LANGCHAIN_ENDPOINT'] = os.getenv('LANGCHAIN_ENDPOINT')
os.environ['LANGCHAIN_TRACING_V2'] = 'true' #true, false
os.environ['LANGCHAIN_PROJECT'] = 'LANG'

if os.getenv('LANGCHAIN_TRACING_V2') == "true":
    print('랭스미스로 추적 중입니다 :', os.getenv('LANGSMITH_API_KEY')[:10])

43b13g4OZS
gpt-4.1-mini
랭스미스로 추적 중입니다 : lsv2_pt_fd


일반적으로 llm.invoke()를 호출하면 한 번에 하나의 입력만 처리함.   
처리해야 하는 요청이 많으면 통신을 계속 호출해야 하므로 비효율적이다.    
하지만 llm.batch()를 사용하면 여러 개의 입력을 한꺼번에 비동기적으로 전송하여 api 호출 횟수를 줄여 응답 속도를 개선할 수 있음.   

In [3]:
llm = AzureChatOpenAI(
    api_key=AZURE_OPENAI_API_KEY,
    azure_endpoint=END_POINT,  
    azure_deployment=MODEL_NAME,          
    api_version="2024-12-01-preview",
    temperature=0.2,
)

In [2]:
user_inputs = [
    "안녕! 오늘 기분 어때?",
    "하늘은 왜 파란색일까?",
    "고양이는 왜 낮잠을 많이 자?",
    "AI가 인간을 대체할까?",
    "서울의 날씨 어때?",
]

### 반복적으로 단일 작업을 요청

In [5]:
# single 처리 및 시간 측정
import time
start_time = time.time()
for i, single_input in enumerate(user_inputs):
    print(f"[입력 {i+1}] {single_input}")
    print(f"[응답 {i+1}] {llm.invoke(single_input).content}\n")
end_time = time.time()
print(f"single 처리 시간: {end_time - start_time}초")

[입력 1] 안녕! 오늘 기분 어때?
[응답 1] 안녕! 나는 항상 기분이 좋아. 너는 오늘 기분이 어때?

[입력 2] 하늘은 왜 파란색일까?
[응답 2] 하늘이 파란색으로 보이는 이유는 대기 중의 빛 산란 현상 때문입니다. 태양빛은 여러 가지 색(파장)의 빛으로 이루어져 있는데, 이 빛이 지구 대기의 분자나 작은 입자에 부딪히면 산란됩니다. 이때 파장이 짧은 파란색 빛이 다른 색에 비해 더 많이 산란되기 때문에 하늘이 파랗게 보이는 것입니다.

좀 더 자세히 설명하자면, 태양빛은 여러 색의 빛이 섞여 있는데, 각각의 빛은 파장이 다릅니다. 파장이 짧은 빛(파란색과 보라색)은 대기 중의 분자에 의해 더 많이 산란되는 경향이 있습니다. 이를 레일리 산란(Rayleigh scattering)이라고 부릅니다. 인간의 눈은 파란색에 더 민감하고, 보라색 빛은 태양빛에 상대적으로 적게 포함되어 있기 때문에 하늘은 주로 파란색으로 인식됩니다.

따라서, 하늘이 파란 이유는 태양빛 중 파란색 빛이 대기 중에서 많이 산란되어 우리 눈에 많이 들어오기 때문입니다.

[입력 3] 고양이는 왜 낮잠을 많이 자?
[응답 3] 고양이가 낮잠을 많이 자는 이유는 여러 가지가 있습니다:

1. **에너지 보존**: 고양이는 본래 사냥을 위해 많은 에너지를 소비하는 동물입니다. 야생에서는 사냥할 때 순간적으로 많은 힘을 쓰기 때문에, 평소에는 에너지를 아끼기 위해 많이 자는 습성이 있습니다.

2. **야행성 습성**: 고양이는 원래 야행성 동물로, 밤에 활동하고 낮에는 휴식을 취하는 경향이 있습니다. 그래서 낮에 많이 자고 밤에 깨어 있는 경우가 많습니다.

3. **성장과 회복**: 잠은 몸의 성장과 회복에 중요한 역할을 합니다. 고양이는 잠을 자면서 몸을 회복하고 면역력을 유지합니다.

4. **환경 적응**: 집에서 키우는 고양이는 사냥할 필요가 없지만, 본능적으로 많은 시간을 자면서 휴식을 취합니다. 또한, 안전하고 편안한 환경에서는 더 많이 자는 경향이 있습니다.

요약하자면

### 배치로 한번에 요청

In [6]:
messages_batch = [[{"role": "user", "content": text}] for text in user_inputs]
print(messages_batch)

start_time = time.time()
responses = llm.batch(messages_batch)
end_time = time.time()

# 결과 출력
for i, r in enumerate(responses):
    print(f"[입력 {i+1}] {user_inputs[i]}")
    print(f"[응답 {i+1}] {r.content}\n")
    
print(f"batch 처리 시간: {end_time - start_time}초")

[[{'role': 'user', 'content': '안녕! 오늘 기분 어때?'}], [{'role': 'user', 'content': '하늘은 왜 파란색일까?'}], [{'role': 'user', 'content': '고양이는 왜 낮잠을 많이 자?'}], [{'role': 'user', 'content': 'AI가 인간을 대체할까?'}], [{'role': 'user', 'content': '서울의 날씨 어때?'}]]
[입력 1] 안녕! 오늘 기분 어때?
[응답 1] 안녕! 나는 항상 기분이 좋아. 너는 오늘 기분이 어때?

[입력 2] 하늘은 왜 파란색일까?
[응답 2] 하늘이 파란색으로 보이는 이유는 대기 중의 빛 산란 현상 때문입니다. 태양빛은 여러 가지 색(파장)을 가진 빛의 혼합인데, 이 빛이 지구 대기의 분자와 작은 입자들에 부딪히면서 산란됩니다.

특히, 파란색 빛은 파장이 짧아서 다른 색에 비해 더 많이 산란되는 성질이 있습니다. 이를 레일리 산란(Rayleigh scattering)이라고 부릅니다. 그래서 태양빛이 대기를 통과할 때 파란색 빛이 사방으로 많이 퍼져 나가고, 우리가 하늘을 볼 때 그 산란된 파란색 빛이 많이 들어와서 하늘이 파랗게 보이는 것입니다.

반면, 일출이나 일몰 때는 태양빛이 대기를 더 길게 통과하면서 파란색 빛은 거의 산란되어 사라지고, 상대적으로 파장이 긴 붉은색이나 주황색 빛이 많이 남아 하늘이 붉게 보이게 됩니다.

[입력 3] 고양이는 왜 낮잠을 많이 자?
[응답 3] 고양이가 낮잠을 많이 자는 이유는 여러 가지가 있습니다:

1. **에너지 보존**: 고양이는 본래 사냥을 위해 에너지를 아껴야 하는 동물입니다. 야생에서는 사냥할 때 많은 에너지를 쓰기 때문에, 사냥하지 않는 시간에는 에너지를 절약하기 위해 많이 잡니다.

2. **야행성 습성**: 고양이는 주로 밤이나 새벽에 활동하는 야행성 동물입니다. 낮에는 활동량이 적고, 밤에 활발히 움직이기 때문에 낮잠을 많이 자는 경향이 있습니다.

3. **성

## RAG를 배치로 요청하기

In [ ]:
from langchain_community.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader("pdf/인터넷서비스_이용약관.pdf", mode="page") 
docs = loader.load()

from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=50,
    separators=["\n\n", "\n", "  ", " "]
)
splits = splitter.split_documents(docs)
    
from langchain_openai import AzureOpenAIEmbeddings
from langchain_community.vectorstores import Chroma, FAISS

AZURE_OPENAI_EMB_API_KEY = os.getenv('AZURE_OPENAI_EMB_API_KEY')
EMB_END_POINT=os.getenv('EMB_END_POINT')
EMB_MODEL_NAME=os.getenv('EMB_MODEL_NAME')

emb = AzureOpenAIEmbeddings(
    model=EMB_MODEL_NAME,                      
    api_key=AZURE_OPENAI_EMB_API_KEY,
    azure_endpoint=EMB_END_POINT,
    api_version="2024-08-01-preview"
)
vectordb = FAISS.from_documents(splits,emb)
retriever = vectordb.as_retriever()

from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system",
     "너는 문서 기반 QA 어시스턴트다. 아래 규칙을 지켜라.\n"
     "1) 제공된 컨텍스트(표/본문)에서만 근거를 찾아 답한다.\n"
     "2) 모르면 모른다고 말한다. 추측 금지.\n"
     "3) 숫자/조건/예외는 정확히 인용하고, 출처 문장 일부를 함께 제시한다.\n"
     "4) 표 내용이라면 행/열 헤더를 함께 언급해 맥락을 명확히 한다.\n"
     "5) 출처 페이지를 함께 제시한다.\n"),
    ("human", "질문: {question}\n\n컨텍스트:\n{context}\n\n한국어로 간결하게 답해.")
])

from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

parser = StrOutputParser()

def format_docs(docs):
    return "\n\n---\n\n".join(d.page_content for d in docs)

rag_chain = (
    {"question": RunnablePassthrough(),   
     "context": retriever | format_docs}
    | prompt
    | llm
    | parser
)

In [14]:
query_list = ["KT 사장님 배달POS의 비용은?", "해지 위약금은 얼마나 해?", "사용량이 많은 사람이 쓰기 좋은 요금제를 추천해줘", "giga wifi에 대해서 간단하게 설명해줘", "가장 비싼 요금제는 뭐야?"]

### 비교1) 순차적으로 처리

In [26]:
start_time = time.time()
for i, query in enumerate(query_list):
    print(f"[질문 {i+1}]{query}")
    print(f"[답변 {i+1}] {rag_chain.invoke(query)}\n")
end_time = time.time()
print(f"single 처리 시간: {end_time - start_time}초")

[질문 1]KT 사장님 배달POS의 비용은?
[답변 1] KT 사장님 배달POS의 비용은 월 22,000원입니다. (출처: "KT 사장님 배달 POS 다양항 배달앱 주문수락과 배달대행 호출을 동시에 처리하는 배달 중개 서비스... 월 22,000원", 페이지 208)

[질문 2]해지 위약금은 얼마나 해?
[답변 2] 해지 위약금은 약정기간과 이용기간에 따라 다릅니다. 예를 들어, 1년 약정의 경우 계약 후 6개월 이내는 0%, 7~9개월 이내는 20%, 10~12개월 이내는 130%의 할인반환금 할인율을 적용해 위약금을 부과합니다. 2년 약정은 계약 후 6개월 이내 0%, 7~12개월 이내 60%, 13~16개월 이내 95%, 17~20개월 이내 140%, 21~24개월 이내 180% 할인율이 적용됩니다. 3년 약정은 계약 후 6개월 이내 0%, 7~12개월 이내 30% 등입니다. 또한, 약정 이외의 단말수에 대해 최근 6개월 평균이용요금에 3을 곱한 금액으로 위약금을 산정하기도 합니다(“약정 이외의 단말수 * 최근 6개월 평균이용요금 * 3”). (출처: 제공 컨텍스트, 페이지 165)

[질문 3]사용량이 많은 사람이 쓰기 좋은 요금제를 추천해줘
[답변 3] 사용량이 많은 사람이 쓰기 좋은 요금제는 기본 제공 용량과 추가 사용 요금을 고려해야 합니다. 컨텍스트에 따르면, KT biz kornet 기본 이용 용량은 10MB이며, 초과 사용 시 월 8,800원(128Kbps~2.048Mbps) 요금이 부과됩니다(출처: "128Kbps~2.048Mbps FDSU,FDSL,HSM,CSU 월 8,800 원", "단말기기 접속 이용고객이 KT biz kornet서버의 보조기억장치 기본 이용 용량(10MB)을 초과하여 사용하는 요금" / 페이지 미상).

또한 무선온라인 전용 접속 요금제 중에서는 기본 20MB를 제공하는 월 5,500원 요금제가 있습니다(출처: "- 5 월 5,500원/ 기본 20MB" / 페이지 미상).

따라서, 대용량 이용자라면 기본 제공 용량

### 비교2) 배치로 처리

In [23]:
start_time = time.time()
results = rag_chain.batch(query_list)
end_time = time.time()
for i, r in enumerate(results):
    print(f"[질문 {i+1}] {query_list[i]}")
    print(f"[답변 {i+1}] {r}\n")

print(f"batch 처리 시간: {end_time - start_time}초")

[질문 1] KT 사장님 배달POS의 비용은?
[답변 1] KT 사장님 배달POS 비용은 월 22,000원입니다. 또한, 서비스 가입 시 최초 1회 2개월간 월정액 요금 전액 할인이 적용됩니다.  
출처: "KT 사장님 배달POS ... 월 22,000원 서비스 가입시 최초 1회 2개월 간 월정액 요금 전액 할인" (본문 중간)  
페이지: 209

[질문 2] 해지 위약금은 얼마나 해?
[답변 2] 해지 위약금은 약정기간과 이용기간에 따라 다릅니다. 할인반환금에 할인율을 적용해 부과하는데, 예를 들어 1년 약정 시 계약 후 6개월 이내는 0%, 7~9개월 이내는 20%, 10~12개월 이내는 130%입니다. 2년 약정은 6개월 이내 0%, 7~12개월 이내 60%, 13~16개월 95%, 17~20개월 140%, 21~24개월 180%입니다. 3년 약정은 6개월 이내 0%, 7~12개월 이내 30%입니다. (출처: 페이지 165) 

또한, 단말수 초과 시 "약정 이외의 단말수 × 최근 6개월 평균이용요금 × 3" 산정식을 통해 실비성 위약금을 부과할 수 있습니다. (출처: 페이지 165)

[질문 3] 사용량이 많은 사람이 쓰기 좋은 요금제를 추천해줘
[답변 3] 사용량이 많은 사람이 쓰기 좋은 요금제로는 '월 5,500원에 기본 20MB 제공' 요금제가 적합합니다. 컨텍스트에 따르면, "기본 20MB 제공에 월 5,500원" 요금제가 있으며, 이는 가장 많은 기본 제공 용량을 포함하는 요금제입니다(페이지 하단 무선온라인 전용 접속 부분).

[질문 4] giga wifi에 대해서 간단하게 설명해줘
[답변 4] GiGA WiFi는 KT에서 제공하는 다양한 WiFi AP 장비와 서비스 브랜드명으로, 최대 10G 속도를 지원하는 인터넷 상품과 함께 사용할 수 있습니다. 예를 들어, GiGA WiFi home, GiGA WiFi home plus, GiGA WiFi Premium 등 여러 종류가 있으며 요금은 49,500원(홈 플러스)에서 165,000원(프리미

### 비교3) 리트리버를 비동기 형식으로 처리하기

In [24]:
import asyncio

async def async_retrieve_all():
    loop = asyncio.get_event_loop()
    tasks = [loop.run_in_executor(None, retriever.get_relevant_documents, q) for q in query_list]
    return await asyncio.gather(*tasks)

async def main():
    retrieved_docs = await async_retrieve_all()

    contexts = ["\n".join([d.page_content for d in docs]) for docs in retrieved_docs]
    messages_batch = [
        [{"role": "user", "content": prompt.format(question=q, context=c)}]
        for q, c in zip(query_list, contexts)
    ]

    responses = llm.batch(messages_batch)

    for q, r in zip(query_list, responses):
        print(f"Q: {q}\nA: {r.content}\n")

start_time = time.time()
await main()
end_time = time.time()
print(f"batch 처리 시간: {end_time - start_time}초")

Q: KT 사장님 배달POS의 비용은?
A: KT 사장님 배달POS 비용은 월 22,000원입니다. 서비스 가입 시 최초 1회 2개월 간 월정액 요금 전액 할인 프로모션도 있습니다.  
출처: "KT 사장님 배달POS ... 월 22,000원 서비스 가입시 최초 1회 2개월 간 월정액 요금 전액 할인" (페이지 209)

Q: 해지 위약금은 얼마나 해?
A: 해지 위약금은 다음과 같습니다.

1. 약정 외 단말 수 * 최근 6개월 평균 이용요금 * 3 (고객 이용기간이 6개월 미만 시 해당 기간 평균 요금 적용)  
2. 장비 임대료 선납 약정의 경우, 약정 기간과 이용 기간에 따라 할인반환금 할인율을 적용해 위약금을 부과함.

예를 들면, 1년 약정일 때  
- 계약 후 6개월 이내에는 위약금 없음(0%)  
- 7~9개월 이내 20%  
- 10~12개월 이내 130% 부과

2년, 3년 약정도 각각 이용 기간별로 할인율이 달라짐(예: 2년 7~12개월 60%, 3년 7~12개월 30%).

*출처: 제공된 컨텍스트 내 할인반환금 할인율 표, 165페이지*

Q: 사용량이 많은 사람이 쓰기 좋은 요금제를 추천해줘
A: 사용량이 많은 사람에게 적합한 요금제는 "스페셜" 요금 할인형으로, 월 3,300원의 할인 혜택을 받을 수 있는 요금제입니다. 또한, 장기 계약 시 4개월 이용료 면제(1,13,25,37개월차)가 제공되어 부담을 줄일 수 있습니다(출처: "요금 할인형" 표, 할인금액 스페셜 월 3,300원, 1년이상 계약 시 면제 혜택, 페이지 1).

Q: giga wifi에 대해서 간단하게 설명해줘
A: GiGA WiFi는 KT에서 제공하는 고속 무선 인터넷 서비스로, 10GiGA 인터넷부터 기가 인터넷(최대 1G)까지 다양한 인터넷 상품과 연계하여 이용할 수 있습니다. 주요 상품으로는 GiGA WiFi home, home plus, premium, premium plus 등이 있으며, 요금은 49,500원부터 165,000원까지 다양합니다. 또한, Gi